# Exercises Week 11-12: Graded exercise session (part B)

**Course**: [Topics in life sciences engineering](https://moodle.epfl.ch/enrol/index.php?id=17061) (BIO-411)

**Professors**:  _Gönczy Pierre_, _Naef Felix_, _McCabe Brian Donal_

In [2]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
import gseapy as gp

from scipy.integrate import odeint
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed
from scipy.stats import beta

**Provide answers in the form of code, figures (if relevant) and short descriptions (in markdown cells) in those notebooks. Submit your notebook to Moodle, please make sure to execute every cell.**

### Exercise 1: Simulation of circadian gene regulation

Similarly to Week 9 exercises, we first describe a system in which a (nuclear) pre-mRNA is transcribed and spliced to produce an mRNA. We can write a 2D ODE (ignoring the degradation of the pre-mRNA):

\begin{eqnarray*}
&&\frac{dP}{dt} = s(t) - \rho P \\
&&\frac{dM}{dt} = \rho P - k(t) M \\
\end{eqnarray*}

Here, $P$ and $M$ denote, respectively, the concentrations of pre-mRNA and mRNA. We will consider time-dependent (circadian) transcription and degradation rates. The frequency is $\omega=\frac{2\pi}{T}$ with $T=24h$. $s(t)$ is the transcription rate, which is now taken either as constant $s(t)=s_{0}$ or as a rhythmic function of time:  

\begin{equation*}
s(t) = s_0 + (1 + \epsilon_s \cos(\omega t)) 
\end{equation*}  
with relative amplitude $0\leq \epsilon_s\leq 1$. Note that here the peak of $s(t)$ is at $t=0$. 
Similarly, we will consider $k(t)$ either as constant $k(t)= k_0$ or a rhythmic function of time:  
\begin{equation*}
k(t)=k_0(1 + \epsilon_k \cos(\omega (t-t_k))
\end{equation*}  
with $0\leq\epsilon_k\leq 1$, and the maximum degradation rate at $t=t_k$.

$\rho$ is the splicing rate of pre-mRNA $P$ to mRNA $M$, and is also taken as a constant.

### Question 1
1. Adapt the code of week 9 exercises to simulate this new system. Modify the interactive widget such that you can vary the new parameters and plot $P(t)$ and $M(t)$.  
2. Keep the degradation rate constant ($\epsilon_k$=0) and vary $k_0$ and $\epsilon_s$. Discuss on how 1) the phase delay between $M(t)$ and $P(t)$ and 2) the relative amplitude of $M(t)$ varies in function of $k_0$. What is the maximal relative amplitude? Make sure you span a relevant range for $k_0^{-1}$ ( *i.e.* from 10 minutes to several hours).
3. Vary $\epsilon_k$ to include rhythmic degradation. Discuss the different effects, in particular, find situations that extend beyond what is possible with a constant half-life. See also [Wang et al., 2018](https://www.pnas.org/doi/10.1073/pnas.1715225115).

### Exercise 2: Circadian (post-)transcriptional regulation of gene expression in mouse liver

We will now analyse published RNA-seq data from mouse liver under an *ad libitum* feeding regimen and in presence of a 12h-12h light-dark cycle ([Atger et al., 2015](https://www.pnas.org/doi/abs/10.1073/pnas.1515308112)). The samples were harvested every 2 hours in four replicates, RNA was extracted and sequenced. Similarly to week 9 exercises on RNA-seq data, gene expression was quantified at the intron and exon levels. Note that data are $log_2$ transformed and normalized (RPKM).

### Question 1
1. Adapt code from week 9 exercises to perform PCA using the 14 following genes (core clock genes and selected clock output genes): ``['Arntl', 'Npas2', 'Clock', 'Cry1', 'Cry2', 'Per1', 'Per2', 'Per3', 'Nr1d1', 'Nr1d2', 'Rora', 'Rorc', 'Tef', 'Dbp']``. First start with only the exons, then only the introns, and eventually both. Describe your observations.
2. Using the *return_amp_phase_pv* function with the $log_2$ data, assess rhythmicity genome-wide (use the p-value to make gene selections). For the selected genes, provide histograms of the peak times, and amplitudes ($log_2$ peak-to-trough). Do this separately for the exons and introns. Do you see what's commonly referred to as the morning and evening waves of genes expression?
3. Which genes are overall the most robustly rhythmic? Is there a theme?
3. Adapting the enrichR code from Week 9 exercises, find biological function or transcriptional programs regulated rhythmically in the mouse liver. Do this separately for the night and the day. Comment.

#### Load the RNA-seq data

In [20]:
dat = pd.read_csv("./GSE73554_WT_AL_Intron_Exon_RFP.txt",sep='\t')
dat.index = dat['Gene_Symbol']
dat = dat.drop(['Gene_Symbol','Gene_Ensembl'], axis=1)
dat.columns = dat.columns.str.split('_', expand = True)
dat.columns.names = ['condition','feeding','feature','time','replicate']

dat.head()

condition          WT                                                    \
feeding            AL                                                     
feature        Intron                                                     
time               00        02        04        06        08        10   
replicate           A         A         A         A         A         A   
Gene_Symbol                                                               
Gnai3       -1.564402 -1.536626 -1.931707 -1.458354 -1.575970 -1.654038   
Cdc45       -3.838233 -4.132913 -4.023130 -3.782926 -3.971205 -3.898270   
Apoh        -1.075680 -1.540220 -1.569142 -1.652602 -1.786800 -1.539642   
Narf        -1.198793 -0.937221 -0.921084 -0.331866 -0.123671 -0.186613   
Cav2        -3.046334 -3.741371 -3.364087 -3.063080 -3.374724 -2.992739   

condition                                            ...             \
feeding                                              ...              
feature                                              ...        RFP   
time               12        14        16        18  ...         04   
replicate           A         A         A         A  ...          D   
Gene_Symbol                                          ...              
Gnai3       -1.550707 -1.564596 -1.211372 -1.654187  ...   4.739938   
Cdc45       -3.837377 -3.907227 -4.002231 -3.837719  ...  -1.783427   
Apoh        -1.434320 -1.655522 -1.499171 -1.411709  ...  10.657651   
Narf        -0.632810 -0.963087 -0.999763 -1.320254  ...   1.788997   
Cav2        -2.985271 -2.991166 -2.954387 -3.052015  ...   0.759856   

condition                                                                      \
feeding                                                                         
feature                                                                         
time                06         08         10         12         14         16   
replicate            D          D          D          D          D          D   
Gene_Symbol                                                                     
Gnai3         4.648414   4.576937   4.891934   5.284247   5.502545   5.115915   
Cdc45        -1.045727  -1.868940  -1.616297   1.552717   0.336330  -1.370050   
Apoh         10.460193  10.623698  10.381588  10.146668  10.232355  10.631922   
Narf          2.660656   2.890531   3.379632   2.924193   2.472548   2.686118   
Cav2          0.403964   0.581452   0.716145   0.742948   0.385111   0.282426   

condition                                     
feeding                                       
feature                                       
time                18         20         22  
replicate            D          D          D  
Gene_Symbol                                   
Gnai3         5.402904   5.119890   4.873039  
Cdc45        -0.308212  -1.218384  -0.541827  
Apoh         10.468625  10.704931  10.591210  
Narf          2.390108   1.941836   2.128767  
Cav2          0.624375   0.806800   0.524357  

[5 rows x 144 columns]

#### Assess rhythmicity

In [22]:
# Function to infer the p-value, phase, amplitude and mean from a time-serie y with size N, period T and sampling Ts. 
def return_amp_phase_pv(y, Ts, T, N):
    
    # we do the harmonic regression using a Fourier series
    t = np.linspace(0.0, N * Ts, N)
    x_fft = np.fft.fft(y)
    freq = np.fft.fftfreq(len(y), d=Ts)
    index, = np.where(np.isclose(freq, 1/T, atol=0.005))
    amp = 4 / N * np.abs(x_fft[index[0]])
    phase = T * np.arctan2(-x_fft[index[0]].imag, x_fft[index[0]].real) / (2 * np.pi)
    mu = 1 / N * x_fft[0].real
    
    #compute the residuals and statistics of the fit (pval)
    x_hat = mu + 0.5 * amp * np.cos(2 * np.pi / T * t - 2 * np.pi * phase / T)
    res = y - x_hat
    sig2_1 = np.var(res)
    sig2 = np.var(y)
    R2 = 1 - sig2_1 / sig2
    p = 3
    pval = 1 - beta.cdf(R2, (p - 1) / 2, (N - p) / 2)
    if phase < 0:
        phase += T
        
    return amp, phase, pval, mu

In [23]:
#Apply the function for all the genes at the intron or exon level and retrieve the amplitude, phase, mean and p-value.

Ts = 2.0 # sampling time 
T = 24 # period
N = 48 # number of samples

amp_intron = []
phase_intron = [] 
pv_intron = []
mu_intron = []

i_pos = dat.columns.get_level_values('feature').isin(['Intron'])

for i, row in dat.iterrows():
    [a, p, pv, mu] = return_amp_phase_pv(np.array(row)[i_pos], Ts, T, N)
    amp_intron.append(a)
    phase_intron.append(p)
    pv_intron.append(pv)
    mu_intron.append(mu)
    
amp_exon = []
phase_exon = [] 
pv_exon = []
mu_exon = []
e_pos = dat.columns.get_level_values('feature').isin(['Exon'])

for i, row in dat.iterrows():
    [a, p, pv, mu]=return_amp_phase_pv(np.array(row)[e_pos], Ts, T, N)
    amp_exon.append(a)
    phase_exon.append(p)
    pv_exon.append(pv)
    mu_exon.append(mu)

#convert the lists to numpy arrays
phase_intron=np.array(phase_intron)
amp_intron=np.array(amp_intron)
pv_intron=np.array(pv_intron)
mu_intron=np.array(mu_intron)

phase_exon=np.array(phase_exon)
amp_exon=np.array(amp_exon)
pv_exon=np.array(pv_exon)
mu_exon=np.array(mu_exon)

### Question 2
1. Study the phase relationship between the pre-mRNA and mRNA for genes that you selected to be rhythmic at both the pre-mRNA and mRNA levels. Use cutoffs that give you several hundred genes.
2. Show representative profiles of genes with short or large delays.
3. Analyze the phase difference in function of the ratio of relative amplitudes of the mRNA and pre-mRNA. What is the expected behavior for genes with constant degradation (see course)?  
4. For genes with a positive difference bewteen 0h and 6h, describe whether these follow the prediction.
5. Identify few outliers and discuss them. Can you say something about how these might be regulated?

In [24]:
#Note that because you fitted the log2 data, the relative amplitudes take the form
relamp_exon = 2**amp_exon[ii] - 2**(-amp_exon[ii])
relamp_intron = 2**amp_intron[ii]- 2**(-amp_intron[ii])